In [ ]:
!pip install transformers accelerate

In [31]:
import pandas as pd
import random
import math
import numpy as np
from scipy.stats import spearmanr
from transformers import AlbertTokenizer, AlbertModel
import argparse as ap

from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/GIL/Mushoom/'#C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\GPT_Answers\

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
gpt_answers = pd.read_csv(f'{path}Datasets/QAC_Val.csv')
gpt_answers = gpt_answers.drop(columns = ["Unnamed: 0"])
gpt_answers.head(6)

,Pregunta,Respuesta GPT,Contexto
0,In which city were the 26th biathlon world cha...,The 26th Biathlon World Championships were hel...,The 26th Biathlon World Championships were hel...
1,Who did Dave Franco play in Scrubs?,Dave Franco played the character Cole in the f...,"David John Franco (born June 12, 1985) is an A..."
2,Which municipalities does the Italian commune ...,Ponzone borders the following municipalities: ...,The province of Alessandria (Italian: provinci...
3,What is the UN Sustainable Development Goal 11...,The provided text does not contain information...,"SDG 4, or Sustainable Development Goal 4, is a..."
4,Is the Arts and Humanities Citation Index stil...,"Yes, the Arts and Humanities Citation Index (A...","The Arts and Humanities Citation Index (AHCI),..."
5,What is the home stadium of FC Zhenis?,The home stadium of FC Zhenis is the K.Munaitp...,FC Zhenis (Kazakh: Жеңіс футбол клубы) is a Ka...


In [5]:
val_ds = pd.read_json(f'{path}Datasets/mushroom.en-val.v2.jsonl', lines = True)
val_ds.head(6)

,id,lang,model_input,model_output_text,model_id,soft_labels,hard_labels,model_output_logits,model_output_tokens
0,val-en-1,EN,What did Petra van Staveren win a gold medal for?,Petra van Stoveren won a silver medal in the 2...,tiiuae/falcon-7b-instruct,"[{'start': 10, 'prob': 0.2, 'end': 12}, {'star...","[[25, 31], [45, 49], [69, 83]]","[-5.5669536591, -11.90533638, -13.0743436813, ...","[Pet, ra, Ġvan, ĠSto, ve, ren, Ġwon, Ġa, Ġsilv..."
1,val-en-2,EN,How many genera does the Erysiphales order con...,The Elysiphale order contains 5 genera.,tiiuae/falcon-7b-instruct,"[{'start': 4, 'prob': 0.2, 'end': 14}, {'start...","[[30, 31]]","[-6.199614048, -13.7564926147, -14.0058326721,...","[The, ĠE, lys, iph, ale, Ġorder, Ġcontains, Ġ,..."
2,val-en-3,EN,Do all arthropods have antennae?,"Yes, all arachnids have antennas. However, not...",tiiuae/falcon-7b-instruct,"[{'start': 0, 'prob': 0.6000000000000001, 'end...","[[0, 3], [9, 18], [63, 70], [78, 87]]","[-4.8190689087, -16.5279369354, -10.1344690323...","[Yes, ,, Ġall, Ġar, ach, n, ids, Ġhave, Ġanten..."
3,val-en-4,EN,When did Chance the Rapper debut?,Chance the rapper debuted in 2011.,tiiuae/falcon-7b-instruct,"[{'start': 18, 'prob': 0.0909090909, 'end': 29...","[[29, 33]]","[-6.2555613518, -10.9392805099, -13.1335763931...","[Ch, ance, Ġthe, Ġrapper, Ġdebuted, Ġin, Ġ, 20..."
4,val-en-5,EN,What is the UN Sustainable Development Goal 11...,The UN's Sustainable City initiative defines a...,tiiuae/falcon-7b-instruct,"[{'start': 0, 'prob': 0.0909090909, 'end': 9},...","[[70, 227]]","[-3.9737114906, -12.9197320938, -7.2846975327,...","[The, ĠUN, ', s, ĠSustainable, ĠCity, Ġinitiat..."
5,val-en-6,EN,What are the four styles of Zhejiang cuisine?,Zhejing cuisine is known for its unique flavor...,tiiuae/falcon-7b-instruct,"[{'start': 0, 'prob': 0.2727272727, 'end': 7},...","[[101, 109], [154, 158], [163, 170], [210, 220...","[-5.0601181984, -13.6542539597, -13.8530063629...","[Z, he, j, ing, Ġcuisine, Ġis, Ġknown, Ġfor, Ġ..."


In [32]:
merge_data = pd.merge(val_ds, gpt_answers, left_on="model_input", right_on="Pregunta")
merge_data = merge_data[['id','model_input', 'model_output_text', 'Respuesta GPT']]
merge_data.head(6)

,id,model_input,model_output_text,Respuesta GPT
0,val-en-1,What did Petra van Staveren win a gold medal for?,Petra van Stoveren won a silver medal in the 2...,Petra van Staveren won a gold medal in the 100...
1,val-en-2,How many genera does the Erysiphales order con...,The Elysiphale order contains 5 genera.,"The Erysiphales order contains one family, Ery..."
2,val-en-3,Do all arthropods have antennae?,"Yes, all arachnids have antennas. However, not...",Not all arthropods have antennae. While most a...
3,val-en-4,When did Chance the Rapper debut?,Chance the rapper debuted in 2011.,"Chance the Rapper debuted with his mixtape ""10..."
4,val-en-5,What is the UN Sustainable Development Goal 11...,The UN's Sustainable City initiative defines a...,The provided text does not contain information...
5,val-en-6,What are the four styles of Zhejiang cuisine?,Zhejing cuisine is known for its unique flavor...,The four styles of Zhejiang cuisine are:\n\n1....


In [33]:
merge_data.iloc[0]['Respuesta GPT'], merge_data.iloc[0]['model_output_text']

('Petra van Staveren won a gold medal in the 100 meter breaststroke at the 1984 Summer Olympics in Los Angeles.',
 'Petra van Stoveren won a silver medal in the 2008 Summer Olympics in Beijing, China.')

In [34]:
merge_data.iloc[1]['Respuesta GPT'], merge_data.iloc[1]['model_output_text']

('The Erysiphales order contains one family, Erysiphaceae. It includes multiple genera, but the exact number is not specified in the provided text.',
 'The Elysiphale order contains 5 genera.')

# Get labels

In [148]:
def get_hard_labels(claims, evidences):
  hard_labels = []
  for c, e in zip(claims, evidences):
    diff = list(set(c.split()) - set(e.split()))
    label = []
    for i in diff:
      start = c.index(i)
      label.append([start, start+len(i)])
    hard_labels.append(label)
  return hard_labels


In [144]:
get_hard_labels([merge_data.iloc[1]['model_output_text']],[merge_data.iloc[1]['Respuesta GPT']])

['genera.', 'Elysiphale', '5']


[[[32, 39], [4, 14], [30, 31]]]

In [149]:
merge_data['hard_labels'] = get_hard_labels(merge_data['model_output_text'], merge_data['Respuesta GPT'])
merge_data.head(6)

,id,model_input,model_output_text,Respuesta GPT,hard_labels,soft_labels
0,val-en-1,What did Petra van Staveren win a gold medal for?,Petra van Stoveren won a silver medal in the 2...,Petra van Staveren won a gold medal in the 100...,"[[25, 31], [45, 49], [10, 18], [78, 84], [69, ...","[{'start': 0, 'prob': 0.6666666, 'end': 4}, {'..."
1,val-en-2,How many genera does the Erysiphales order con...,The Elysiphale order contains 5 genera.,"The Erysiphales order contains one family, Ery...","[[32, 39], [4, 14], [30, 31]]","[{'start': 4, 'prob': 0.6666666, 'end': 8}, {'..."
2,val-en-3,Do all arthropods have antennae?,"Yes, all arachnids have antennas. However, not...",Not all arthropods have antennae. While most a...,"[[59, 62], [63, 70], [54, 58], [34, 42], [0, 4...","[{'start': 23, 'prob': 0.333333, 'end': 32}, {..."
3,val-en-4,When did Chance the Rapper debut?,Chance the rapper debuted in 2011.,"Chance the Rapper debuted with his mixtape ""10...","[[11, 17], [29, 34], [26, 28]]","[{'start': 0, 'prob': 1, 'end': 4}, {'start': ..."
4,val-en-5,What is the UN Sustainable Development Goal 11...,The UN's Sustainable City initiative defines a...,The provided text does not contain information...,"[[169, 174], [70, 78], [138, 147], [216, 228],...","[{'start': 44, 'prob': 1, 'end': 48}, {'start'..."
5,val-en-6,What are the four styles of Zhejiang cuisine?,Zhejing cuisine is known for its unique flavor...,The four styles of Zhejiang cuisine are:\n\n1....,"[[52, 59], [93, 96], [101, 109], [355, 363], [...","[{'start': 5, 'prob': 0.333333, 'end': 8}, {'s..."


# Evaluation

In [169]:
def recompute_hard_labels(soft_labels):
    """optionally, infer hard labels from the soft labels provided"""
    hard_labels = []
    prev_end = -1
    for start, end in (
        (lbl['start'], lbl['end'])
        for lbl in sorted(soft_labels, key=lambda span: (span['start'], span['end']))
        if lbl['prob'] > 0.5
    ):
        if start == prev_end:
            hard_labels[-1][-1] = end
        else:
            hard_labels.append([start, end])
        prev_end = end
    return hard_labels


def infer_soft_labels(hard_labels):
    """reformat hard labels into soft labels with prob 1"""
    return [
        {
            'start': start,
            'end': end,
            'prob': 1.0,
        }
        for start, end in hard_labels
    ]


def load_jsonl_file_to_records(filename, is_ref=True):
    """read data from a JSONL file and format that as a `pandas.DataFrame`.
    Performs minor format checks (ensures that some labels are present,
    optionally compute missing labels on the fly)."""
    df = pd.read_json(filename, lines=True)
    if not is_ref:
        assert ('hard_labels' in df.columns) or ('soft_labels' in df.columns), \
            f'File {filename} contains no predicted label!'
        if 'hard_labels' not in df.columns:
            df['hard_labels'] = df.soft_labels.apply(recompute_hard_labels)
        elif 'soft_labels' not in df.columns:
            df['soft_labels'] = df.hard_labels.apply(infer_soft_labels)
    # adding an extra column for convenience
    columns = ['id', 'soft_labels', 'hard_labels']
    if is_ref:
        df['text_len'] = df.model_output_text.apply(len)
        columns += ['text_len']
    df = df[columns]
    return df.sort_values('id').to_dict(orient='records')

def score_iou(ref_dict, pred_dict):
    """computes intersection-over-union between reference and predicted hard labels, for a single datapoint.
    inputs:
    - ref_dict: a gold reference datapoint,
    - pred_dict: a model's prediction
    returns:
    the IoU, or 1.0 if neither the reference nor the prediction contain hallucinations
    """
    # ensure the prediction is correctly matched to its reference
    assert ref_dict['id'] == pred_dict['id']
    # convert annotations to sets of indices
    ref_indices = {idx for span in ref_dict['hard_labels'] for idx in range(*span)}
    pred_indices = {idx for span in pred_dict['hard_labels'] for idx in range(*span)}
    # avoid division by zero
    if not pred_indices and not ref_indices: return 1.
    # otherwise compute & return IoU
    return len(ref_indices & pred_indices) / len(ref_indices | pred_indices)

def score_cor(ref_dict, pred_dict):
    """computes Spearman correlation between predicted and reference soft labels, for a single datapoint.
    inputs:
    - ref_dict: a gold reference datapoint,
    - pred_dict: a model's prediction
    returns:
    the Spearman correlation, or a binarized exact match (0.0 or 1.0) if the reference or prediction contains no variation
    """
    # ensure the prediction is correctly matched to its reference
    assert ref_dict['id'] == pred_dict['id']
    # convert annotations to vectors of observations
    ref_vec = [0.] * ref_dict['text_len']
    pred_vec = [0.] * ref_dict['text_len']
    for span in ref_dict['soft_labels']:
        for idx in range(span['start'], span['end']):
            ref_vec[idx] = span['prob']
    for span in pred_dict['soft_labels']:
        for idx in range(span['start'], span['end']):
            pred_vec[idx] = span['prob']
    # constant series (i.e., no hallucination) => cor is undef
    if len({round(flt, 8) for flt in pred_vec}) == 1 or len({round(flt, 8) for flt in ref_vec}) == 1 :
        return float(len({round(flt, 8) for flt in ref_vec}) == len({round(flt, 8) for flt in pred_vec}))
    # otherwise compute Spearman's rho
    return spearmanr(ref_vec, pred_vec).correlation

def main(ref_dicts, pred_dicts, output_file=None):
    assert len(ref_dicts) == len(pred_dicts)
    ious = np.array([score_iou(r, d) for r, d in zip(ref_dicts, pred_dicts)])
    cors = np.array([score_cor(r, d) for r, d in zip(ref_dicts, pred_dicts)])
    if output_file is not None:
        with open(output_file, 'w') as ostr:
            print(f'IoU: {ious.mean():.8f}', file=ostr)
            print(f'Cor: {cors.mean():.8f}', file=ostr)
    return ious, cors

In [165]:
probs_dic = {0:0.333333, 1:0.6666666, 2:1}
probas_list = []
for proba in merge_data.hard_labels:
    lista_aux=[]
    for i in proba:
        prob = random.randint(0,2)
        proba_num = probs_dic[prob]
        probabilidades = {'start':i[0], 'prob':proba_num, 'end':i[-1]}
        lista_aux.append(probabilidades)
    probas_list.append(lista_aux)

merge_data['soft_labels'] = probas_list
merge_data.head(6)

,id,model_input,model_output_text,Respuesta GPT,hard_labels,soft_labels
0,val-en-1,What did Petra van Staveren win a gold medal for?,Petra van Stoveren won a silver medal in the 2...,Petra van Staveren won a gold medal in the 100...,"[[25, 31], [45, 49], [10, 18], [78, 84], [69, ...","[{'start': 25, 'prob': 0.333333, 'end': 31}, {..."
1,val-en-2,How many genera does the Erysiphales order con...,The Elysiphale order contains 5 genera.,"The Erysiphales order contains one family, Ery...","[[32, 39], [4, 14], [30, 31]]","[{'start': 32, 'prob': 1, 'end': 39}, {'start'..."
2,val-en-3,Do all arthropods have antennae?,"Yes, all arachnids have antennas. However, not...",Not all arthropods have antennae. While most a...,"[[59, 62], [63, 70], [54, 58], [34, 42], [0, 4...","[{'start': 59, 'prob': 0.6666666, 'end': 62}, ..."
3,val-en-4,When did Chance the Rapper debut?,Chance the rapper debuted in 2011.,"Chance the Rapper debuted with his mixtape ""10...","[[11, 17], [29, 34], [26, 28]]","[{'start': 11, 'prob': 0.6666666, 'end': 17}, ..."
4,val-en-5,What is the UN Sustainable Development Goal 11...,The UN's Sustainable City initiative defines a...,The provided text does not contain information...,"[[169, 174], [70, 78], [138, 147], [216, 228],...","[{'start': 169, 'prob': 0.6666666, 'end': 174}..."
5,val-en-6,What are the four styles of Zhejiang cuisine?,Zhejing cuisine is known for its unique flavor...,The four styles of Zhejiang cuisine are:\n\n1....,"[[52, 59], [93, 96], [101, 109], [355, 363], [...","[{'start': 52, 'prob': 1, 'end': 59}, {'start'..."


In [166]:
merge_data.to_json(f'{path}Datasets/test_ds_prueba_jsonl.jsonl', orient="records", lines = True)

In [167]:
test_df = load_jsonl_file_to_records(f'{path}Datasets/test_ds_prueba_jsonl.jsonl')
val_df = load_jsonl_file_to_records(f'{path}Datasets/mushroom.en-val.v2.jsonl')

In [170]:
ious, cors = main(val_df, test_df)
print(f'IoU: {ious.mean():.8f}')
print(f'Cor: {cors.mean():.8f}')

IoU: 0.34704504
Cor: 0.31591253


Con el random:

- IoU: 0.15334043
- Cor: 0.05996364